This work started with the question:  how many classes until my friend Andrew, who teaches at Western,
is exposed to COVID in class?  A Georga Tech web site [estimates the risk of someone with COVID)[https://covid19risk.biosci.gatech.edu]
attending one even.  How does that risk change over a series. of events.

Start wth imports.  I used `numpy` which implements arrays and operatiosn on arrays.

Plotting is based on (examples from geeks for geeks)[https://www.geeksforgeeks.org/plot-mathematical-expressions-in-python-using-matplotlib/]

In [ ]:
# Import libraries
import random
import matplotlib.pyplot as plt
import numpy as np

Some assumptions for the model

In [ ]:
CLASSES=20			# Number of classes in a semester
SEMESTERS=750		# Number of semesters that we run when modeling.
PROBABILITIES=(0.5, 0.25, 0.1)	# Exposure probabilities we are interested in.
XAXIS = np.arange(1,CLASSES+1,1)	# X axis of events [1,CLASSES]
FIGSIZE = (20, 10)

I didn't know how to calculate probability but I do know how to model that.  

First, model the probability of exposuer.  In each semester, there is no exposure until the class in which
an infected person attends.  That and all subsequent classes are counted as exposed.

In [ ]:
#
# Multiple ways of modeling if a condition will or will not be met over a number of events.
#
def will_model (probabilty: float, classCount: int, iterations: int=500):
	"""Calculate a condition being met over a number of events.
		If a condition is met for event 'i', it applies to all events > 'i'  
	Parametes:
		probability - the probabilty of the condition
		classCount - number of events.
	Returns
		np array of probability for event.	
		"""
	y = np.zeros(classCount)
	for i in range(iterations):										# Run the specified number of simulations
		isEvent = False												# start with the not the condition
		for t in range(classCount):										# Step through the specified number of events.
			isEvent = isEvent or random.random() < probabilty		# At each event, the condition may happen.  If it happened at a previous event, it still applies to this event.
			if isEvent:												# so, if it applies to this event, 
				y[t] += 1											# then CLASSES the condition true at this event.
	r = y / iterations												# Normalize to probabilities.
	return r

Modeling the probability of no exposuer is done by counting each class as not exposed until an
infected person attends.  

In [ ]:
def wont_model (probabilty: float, classCount: int, iterations: int=500):
	"""Model condition not met over a number of events.  
		In each run, each event is considered to not have condition until the condition 
		is met.  Once condition is met, all subsequent runs are consider to have condition.
	Parametes:
		probability - the probabilty of the condition
		classCount - number of events.
		iterations - number of sitmulation iterations to run.
	Returns
		np array of probability for event.	
		"""
	y = np.zeros(classCount)
	for i in range(iterations):										# Run the specified number of simulations
		isEvent = False												# start with the not the condition
		for t in range(classCount):										# Step through the specified number of events.
			if random.random() < probabilty:						# At each event, the condition may happen.  
				break												# If it happens, we are done simulating this run and exit the loop
			y[t] += 1												# Otherwise, CLASSES this event as not having the condition.
	r = y / iterations												# Normalize to probabilities.
	return r

Try modeling and plot the results

In [ ]:
fig = plt.figure(figsize = FIGSIZE)
for p in PROBABILITIES:
	plt.plot(XAXIS, wont_model(p, CLASSES), "+-", label=f"{p} won't model")
	plt.plot(XAXIS, will_model(p, CLASSES), "+-", label=f"{p} will modeled")

plt.grid(alpha =.6, linestyle ='--') 
plt.yticks(np.linspace(0,1,11))
plt.xticks(XAXIS)
plt.legend()
plt.show()

# Calculation
I don't know how to caculate the probability of getting exposed but I do know that the probability of
not getting exposed is product of the probability of not getting exposed each class.  if `p` is the
probability of exposure then the probability of not getting exposed is `1-p`.  And the probability
of not getting exposed after `i` classe is

	(1-p)^i

In [ ]:
def wont_calculated(probability, classCount):
	"""Calculate something not happening over a number of events.  
	Parametes:
		probability - the probabilty of the condition
		classCount - number of events.
	Returns
		np array of probability for event.	
		"""
	n = 1.0 - probability									# Probability it won't happen, 'n', is opposite of the probibility it will
	y = np.zeros(classCount)				
	for classNum in range(1, classCount+1):
		y[classNum-1] = pow(n, classNum)					# For event 'i', the probability is n^eventNum
	return y

The probability of not getting exposed is `1 - probability of getting exposed`.

Using this feels like a bit of a trick.  It seems like there shold be a direct way to calculate the probability
of something happening over a series of events.  However, taking the inverse of it not happening is a very
simple callcuation.

In [ ]:
def will_calculated(probability, classCount):
	"""Calculate that an event will happen"""
	w = wont_calculated(probability, classCount)			# First, calcualte the probability that it won't happen
	return 1.0 - w											# Probability of getting exposed (operates on every array element)

Now plot the models and claculations.

In [ ]:
fig = plt.figure(figsize = FIGSIZE)
for p in PROBABILITIES:
	plt.plot(XAXIS, wont_model(p, CLASSES), "+-", label=f"{p} won't model")
	plt.plot(XAXIS, wont_calculated(p, CLASSES), "^-", label=f"{p} won'tcalculated")
	plt.plot(XAXIS, will_model(p, CLASSES), "+-", label=f"{p} will modeled")
	plt.plot(XAXIS, will_calculated(p, CLASSES), "o-", label=f"{p} will calculated")

plt.grid(alpha =.6, linestyle ='--') 
plt.yticks(np.linspace(0,1,11))
plt.xticks(XAXIS)
plt.legend()
plt.show()